Codes for spliting training and validation set for cross-validation.  
- Use `seq_id` and `common_name` to create groups.
- Write function to Split feature dataframe on groups using `GroupKFold` of sklearn.
- reference: [Sklearn GroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html)

# Set up
Import libraries

In [ ]:
import pandas as pd
import json
import pickle
import pprint # pretty printing dictionaries
import re
import os
import shutil #save images
from tqdm import tqdm #progress bar
import cv2
import matplotlib.pyplot as plt

# for k-fold cross-validation
from sklearn.model_selection import GroupKFold

##Mount google drive and GCS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Install Cloud Storage FUSE.
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2659  100  2659    0     0  16302      0 --:--:-- --:--:-- --:--:-- 16413
OK
47 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 n

In [ ]:
# Mount a Cloud Storage bucket or location
mount_path = "281-project-d5d834b8-2d7c-11ef-91d5-b89a2a9d8518"
local_path = f"/mnt/gs/{mount_path}"

!mkdir -p {local_path}
!gcsfuse --implicit-dirs {mount_path} {local_path}

{"timestamp":{"seconds":1722092764,"nanos":994220531},"severity":"INFO","message":"Start gcsfuse/2.3.2 (Go version go1.22.4) for app \"\" using mount point: /mnt/gs/281-project-d5d834b8-2d7c-11ef-91d5-b89a2a9d8518\n"}
{"timestamp":{"seconds":1722092764,"nanos":995149538},"severity":"INFO","message":"GCSFuse mount command flags: {\"AppName\":\"\",\"Foreground\":false,\"ConfigFile\":\"\",\"MountOptions\":{},\"DirMode\":493,\"FileMode\":420,\"Uid\":-1,\"Gid\":-1,\"ImplicitDirs\":true,\"OnlyDir\":\"\",\"RenameDirLimit\":0,\"IgnoreInterrupts\":true,\"CustomEndpoint\":null,\"BillingProject\":\"\",\"KeyFile\":\"\",\"TokenUrl\":\"\",\"ReuseTokenFromUrl\":true,\"EgressBandwidthLimitBytesPerSecond\":-1,\"OpRateLimitHz\":-1,\"SequentialReadSizeMb\":200,\"AnonymousAccess\":false,\"MaxRetrySleep\":30000000000,\"StatCacheCapacity\":20460,\"StatCacheTTL\":60000000000,\"TypeCacheTTL\":60000000000,\"KernelListCacheTtlSeconds\":0,\"HttpClientTimeout\":0,\"MaxRetryDuration\":-1000000000,\"RetryMultiplier

In [ ]:
input_path = os.path.join(local_path, 'feature_matrices')

In [ ]:
#test mounting
os.listdir(input_path)

['test_features_matrix.pkl',
 'train_features_matrix.pkl',
 'val_features_matrix.pkl']

# Prepare Pickles for Image Labels Dataframes
Each dataframe contains image filename, common name and group id.
- image: filename on GCP storage. (e.g. N25_100EK113_03030539.JPG)
- label: classication targets (e.g. Peccary)
- group id: identifyer for unique image sequence, by joining `common_name` and `seq_num` columns.

## Train Set Labels

In [ ]:
# train_csv = '/content/drive/MyDrive/w281_Project/0_data/train_downsampled.csv'

# df_train = pd.read_csv(train_csv)

# df_train['group_id'] = df_train['common_name'] + '-' + df_train['seq_id']

# df_train_label = df_train[['filename_new', 'common_name', 'group_id']]
# df_train_label.columns = ['image', 'label', 'group']

# # set image filename as index
# df_train_label.set_index(keys='image', drop=True, inplace=True, append=False, verify_integrity=True)
# df_train_label.head()

In [ ]:
# set_name = 'train'
# output_path = os.path.join(local_path, 'features')

# #save the feature to google drive
# pickle.dump(df_train_label, open(output_path+'/' + set_name + '_label.pkl', 'wb'))

In [ ]:
#load the feature from file in drive
df_train_label = pickle.load(open(output_path+'/'+set_name+'_label.pkl', 'rb'))
df_train_label.head()

## Validation Set Labels

In [ ]:
# val_csv = '/content/drive/MyDrive/w281_Project/0_data/val_downsampled.csv'

# df_val = pd.read_csv(val_csv)

# df_val['group_id'] = df_val['common_name'] + '-' + df_val['seq_id']

# df_val_label = df_val[['filename_new', 'common_name', 'group_id']]
# df_val_label.columns = ['image', 'label', 'group']

# # set image filename as index
# df_val_label.set_index(keys='image', drop=True, inplace=True, append=False, verify_integrity=True)
# df_val_label.head()

,label,group
image,,
N25_101EK113_04150393.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150391.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150396.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150395.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150392.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2


In [ ]:
# # train, test or validation
# set_name = 'validation'
# output_path = os.path.join(local_path, 'features')

# #save the feature to google drive
# pickle.dump(df_val_label, open(output_path+'/' + set_name + '_label.pkl', 'wb'))

In [ ]:
#load the feature from file in drive
df_val_label = pickle.load(open(output_path+'/'+set_name+'_label.pkl', 'rb'))
df_val_label.head()

,label,group
image,,
N25_101EK113_04150393.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150391.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150396.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150395.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2
N25_101EK113_04150392.JPG,Peccary,Peccary-398e2210-7160-11ec-85f0-5cf3706028c2


## Test Set Labels

In [ ]:
# test_csv = '/content/drive/MyDrive/w281_Project/0_data/test_downsampled.csv'

# df_test = pd.read_csv(test_csv)

# df_test['group_id'] = df_test['common_name'] + '-' + df_test['seq_id']

# df_test_label = df_test[['filename_new', 'common_name', 'group_id']]
# df_test_label.columns = ['image', 'label', 'group']

# # set image filename as index
# df_test_label.set_index(keys='image', drop=True, inplace=True, append=False, verify_integrity=True)
# df_test_label.head()

,label,group
image,,
N29_105EK113_04060336.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060328.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060331.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060326.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060335.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2


In [ ]:
# # train, test or validation
# set_name = 'test'
# output_path = os.path.join(local_path, 'features')

# #save the feature to google drive
# pickle.dump(df_test_label, open(output_path+'/' + set_name + '_label.pkl', 'wb'))

In [ ]:
#load the feature from file in drive
df_test_label = pickle.load(open(output_path+'/'+set_name+'_label.pkl', 'rb'))
df_test_label.head()

,label,group
image,,
N29_105EK113_04060336.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060328.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060331.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060326.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2
N29_105EK113_04060335.JPG,Peccary,Peccary-3962c177-7160-11ec-9a5e-5cf3706028c2


# Grouped K-fold Cross-Validation Split

Create train and validation sets without overlapping image sequences for k-fold cross-validation.

- Use `seq_num` and `common_name` as group
- reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html


## Load Train and Validation Labels

In [ ]:
def load_image_labels(input_path):
    train_lb = pickle.load(open(input_path + '/' + 'train_label'  + '.pkl', 'rb'))
    val_lb = pickle.load(open(input_path + '/' + 'validation_label' + '.pkl', 'rb'))
    test_lb = pickle.load(open(input_path + '/' + 'test_label' + '.pkl', 'rb'))

    return train_lb, val_lb, test_lb

In [ ]:
#load the labels from file in drive
df_train_label = load_image_labels(input_path)[0]
df_val_label = load_image_labels(input_path)[1]
df_test_label = load_image_labels(input_path)[2]


## Load Train and Validation Features

In [ ]:
def load_feature_matrices(input_path):
    train_fm = pickle.load(open(input_path + '/' + 'train_features_matrix'  + '.pkl', 'rb'))
    val_fm = pickle.load(open(input_path + '/' + 'val_features_matrix' + '.pkl', 'rb'))
    test_fm = pickle.load(open(input_path + '/' + 'test_features_matrix' + '.pkl', 'rb'))

    return train_fm, val_fm, test_fm

In [ ]:
# loading feature matrics for each set
x_train = load_feature_matrices(input_path)[0]
x_val = load_feature_matrices(input_path)[1]
x_test = load_feature_matrices(input_path)[2]

In [ ]:
# get indices for feature matrices
train_fm_ind = pickle.load(open(local_path + '/features' + '/train_sift_std'  + '.pkl', 'rb'))
val_fm_ind = pickle.load(open(local_path + '/features' + '/validation_sift_std' + '.pkl', 'rb'))
test_fm_ind = pickle.load(open(local_path + '/features' + '/test_sift_std' + '.pkl', 'rb'))

In [ ]:
# create dataframe with image filename as the indices
df_train = pd.DataFrame({'pca': [pca_vector for pca_vector in x_train]}, index = list(train_fm_ind.keys()))
df_val = pd.DataFrame({'pca': [pca_vector for pca_vector in x_val]}, index = list(val_fm_ind.keys()))
df_test = pd.DataFrame({'pca': [pca_vector for pca_vector in x_test]}, index = list(test_fm_ind.keys()))

In [ ]:
# join feature dataframes with labels and groups on the indices
# puting df_{set}_label on the left to preserve suffled image orders
df_train = df_train_label.join(df_train, on=None, how='left', validate='one_to_one')
df_val = df_val_label.join(df_val, on=None, how='left', validate='one_to_one')
df_test = df_test_label.join(df_test, on=None, how='left', validate='one_to_one')

## Combine train and validation feature dataframes



In [ ]:
# combine train and val dataframes
df = pd.concat([df_train, df_val], axis=0)

# Grouped K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.ensemble import RandomForestClassifier

# X, y, and groups are your data and group labels
# specify feature columns
X = df['pca']
# label coluns
y = df['label']
# read group from labels
groups =  df['group']
# specify splits
splits = 3
# model
model = RandomForestClassifier()
# parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# grouped k-fold
gkf = GroupKFold(n_splits=splits)

grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           cv=gkf,
                           scoring='accuracy')

# Fit the model
grid_search.fit(X, y, groups=groups)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)
